In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
from pyproj import Proj, transform
import folium
import warnings
warnings.filterwarnings('ignore')
import re
from shapely.geometry import Point, Polygon, MultiPolygon
import pickle
from haversine import haversine

In [2]:
# pkl 파일 로드
with open('../restaurant_closing.pkl', 'rb') as f:
    restaurant_closing = pickle.load(f)

In [3]:
# pkl 파일 로드
with open('../restaurant.pkl', 'rb') as f:
    restaurant = pickle.load(f)

In [ ]:
# dong pkl 파일 로드
with open('../temp(dong).pkl', 'rb') as f:
    dong = pickle.load(f)   
dong.reset_index(inplace=True)

In [33]:
# 신상권
new = pd.read_csv('../../dataSet/신_상권영역.csv')
# 구상권
old = pd.read_csv('../../dataSet/구_상권영역.csv')

,STDR_YM_CD,TRDAR_SE_C,TRDAR_SE_1,TRDAR_NO,TRDAR_NM,XCNTS_VALU,YDNTS_VALU,SIGNGU_CD,ADSTRD_CD,WKT
0,202112,A,골목상권,2110753,시흥동 은행나무사거리,191934,438813,11545,11545710,POLYGON Z ((126.90968969556398 37.446346796617...
1,202112,A,골목상권,2110427,노원 을지대학교병원,206265,459431,11350,11350611,POLYGON Z ((127.07133835784109 37.633484281881...
2,202112,A,골목상권,2110755,동일여자고등학교,192300,439070,11545,11545680,POLYGON Z ((126.91459483090695 37.450630798710...
3,202112,A,골목상권,2110739,시흥3동 우편취급국,191289,438014,11545,11545690,POLYGON Z ((126.90271679201938 37.440265001668...
4,202112,A,골목상권,2111017,송파나루역 3번,210222,445516,11710,11710562,POLYGON Z ((127.11711377473586 37.508466954813...
...,...,...,...,...,...,...,...,...,...,...
1666,202112,U,관광특구,1001493,동대문패션타운 관광특구,200974,451975,11140,11140590,POLYGON Z ((127.01759028830969 37.569497951283...
1667,202112,U,관광특구,1001494,종로·청계 관광특구,199768,452273,11110,11110615,POLYGON Z ((126.97776556419633 37.570093847004...
1668,202112,U,관광특구,1001495,잠실 관광특구,210191,446339,11710,11710566,POLYGON Z ((127.09307351793956 37.510512512284...
1669,202112,U,관광특구,1001496,강남 마이스 관광특구,205310,445727,11680,11680580,POLYGON Z ((127.06027547260396 37.513903341286...


In [11]:
'''
 - 멀티 폴리곤 데이터 때문에 보류
 - 예시) old['WKT'][72]
# 신상권
new = pd.read_csv('../../dataSet/신_상권영역.csv')
# 구상권
old = pd.read_csv('../../dataSet/구_상권영역.csv')
# poly 데이터 앞뒤 자른 후 리스트로 저장
# 신상권
for i in range(0, len(new)):
    new['WKT'][i] = new['WKT'][i][12:][:-2].split(',') # i번째 poly 데이터 앞뒤 인덱싱 후 온점 구분자로 리스트로 나누기
    temp = new['WKT'][i]
    for j in range(0,len(temp)):
        temp[j] = temp[j][:-2].strip().split(' ') # 나눈 리스트 끝에 있는 불필요한 숫자 0 제거 후 공백으로 분리 후 리스트로 리턴
    new['WKT'][i] = temp # i번째 데이터에 담기

# 신상권 문자열데이터 플롯형으로 변경 후 튜플로 변경
for i in range(0, len(new)):
    for j in range(0, len(new['WKT'][i])):
        for k in range(0,2):
            if(re.search(r'[(]\w', new['WKT'][i][j][k]) == None):
                new['WKT'][i][j][k] = float(new['WKT'][i][j][k])
            else:
                new['WKT'][i][j][k] = float(new['WKT'][i][j][k][1:])
        new['WKT'][i][j] = tuple(new['WKT'][i][j])


# 구상권
old = pd.read_csv('../../dataSet/구_상권영역.csv')
for i in range(0, len(old)):
    old['WKT'][i] = old['WKT'][i][12:][:-2].split(',') # i번째 poly 데이터 앞뒤 인덱싱 후 온점 구분자로 리스트로 나누기
    temp = old['WKT'][i]
    for j in range(0,len(temp)):
        temp[j] = temp[j][:-2].strip().split(' ') # 나눈 리스트 끝에 있는 불필요한 숫자 0 제거 후 공백으로 분리 후 리스트로 리턴
    old['WKT'][i] = temp # i번째 데이터에 담기
    
# 구상권 문자열데이터 플롯형으로 변경
for i in range(0, len(old)):
    for j in range(0, len(old['WKT'][i])):
        for k in range(0,2):
            try:
                if(re.search(r'[(]\w', old['WKT'][i][j][k]) == None):
                    old['WKT'][i][j][k] = float(old['WKT'][i][j][k])
                else:
                    old['WKT'][i][j][k] = float(old['WKT'][i][j][k][1:])
                    print('{0},{1},{2}'.format(i,j,k))
            except:
                print('{0},{1},{2}'.format(i,j,k))
                pass
        old['WKT'][i][j] = tuple(old['WKT'][i][j])
'''

"\n - 멀티 폴리곤 데이터 때문에 보류\n - 예시) old['WKT'][72]\n# 구상권\nold = pd.read_csv('../../dataSet/구_상권영역.csv')\nfor i in range(0, len(old)):\n    old['WKT'][i] = old['WKT'][i][12:][:-2].split(',') # i번째 poly 데이터 앞뒤 인덱싱 후 온점 구분자로 리스트로 나누기\n    temp = old['WKT'][i]\n    for j in range(0,len(temp)):\n        temp[j] = temp[j][:-2].strip().split(' ') # 나눈 리스트 끝에 있는 불필요한 숫자 0 제거 후 공백으로 분리 후 리스트로 리턴\n    old['WKT'][i] = temp # i번째 데이터에 담기\n    \n# 구상권 문자열데이터 플롯형으로 변경\nfor i in range(0, len(old)):\n    for j in range(0, len(old['WKT'][i])):\n        for k in range(0,2):\n            try:\n                if(re.search(r'[(]\\w', old['WKT'][i][j][k]) == None):\n                    old['WKT'][i][j][k] = float(old['WKT'][i][j][k])\n                else:\n                    old['WKT'][i][j][k] = float(old['WKT'][i][j][k][1:])\n                    print('{0},{1},{2}'.format(i,j,k))\n            except:\n                print('{0},{1},{2}'.format(i,j,k))\n                pass\n        old['WKT'][i][

In [4]:
# 좌표변환 함수
from pyproj import Proj
from pyproj import transform

WGS84 = { 'proj':'latlong', 'datum':'WGS84', 'ellps':'WGS84', }

# conamul
TM127 = { 'proj':'tmerc', 'lat_0':'38N', 'lon_0':'127.0028902777777777776E',
   'ellps':'bessel', 'x_0':'200000', 'y_0':'500000', 'k':'1.0'}

# naver
TM128 = { 'proj':'tmerc', 'lat_0':'38N', 'lon_0':'128E', 'ellps':'bessel',
   'x_0':'400000', 'y_0':'600000', 'k':'0.9999'}

# GRS80(Geodetic Reference System 1980：측지 기준계 1980)의 타원체
GRS80 = { 'proj':'tmerc', 'lat_0':'38', 'lon_0':'127', 'k':1, 'x_0':200000,
    'y_0':600000, 'ellps':'GRS80', 'units':'m' }

# EPSG5181(다음카카오)
EPSG5181 = { 'proj':'tmerc', 'lat_0':'38', 'lon_0':'127', 'k':1, 'x_0':200000,
    'y_0':500000, 'ellps':'GRS80', 'units':'m'
}


def wgs84_to_tm128(longitude, latitude):
   return transform( Proj(**WGS84), Proj(**TM128), longitude, latitude )

def tm128_to_wgs84(x, y):
   return transform( Proj(**TM128), Proj(**WGS84), x, y )

def wgs84_to_tm127(longitude, latitude):
   return map(lambda x:2.5*x,
        transform( Proj(**WGS84), Proj(**TM127), longitude, latitude ))

def tm127_to_wgs84(x, y):
   return transform( Proj(**TM127), Proj(**WGS84), x/2.5, y/2.5 )

def grs80_to_wgs84(x, y):
   return transform( Proj(**GRS80), Proj(**WGS84), x, y )

def EPSG5181_to_wgs84(x, y):
   return transform( Proj(**EPSG5181), Proj(**WGS84), x, y )

def wgs84_to_cyworld(longitude, latitude):
   x_min = 4456260.72
   y_min = 1161720.00
   long_min = 123.78323
   lat_min = 32.27345
   max_grid_length = 112721.92
   x = (longitude-long_min)*max_grid_length/3.1308 + x_min
   y = (latitude-lat_min)*max_grid_length/3.1308 + y_min
   return x, y

def cyworld_to_wgs84(x, y):
   x_min = 4456260.72;
   y_min = 1161720.00;
   long_min = 123.78323;
   lat_min = 32.27345;
   max_grid_length = 112721.92;
   longitude = long_min + (x-x_min)*3.1308 / max_grid_length;
   latitude = lat_min + (y-y_min)*3.1308 / max_grid_length;
   return longitude, latitude

In [ ]:
def inside(polygon, x, y):
    """
    폴리곤 영역안에 x좌표와 y좌표값에 해당하는 위치가 포함되는지 여부 boolean값으로 리턴
    """
    poly_data = polygon
    poly = Polygon(poly_data)
    coordinate = Point(x, y)
    return coordinate.within(poly)

### 신상권, 구상권 경도 위도 변경
 - 함수를 활용하여 wgs84로 변경

In [4]:
# 경도위도 변경
new['y'] = 0
new['x'] = 0
for i in range(0, len(new['XCNTS_VALU'])):
    new['y'][i], new['x'][i] = EPSG5181_to_wgs84(new['XCNTS_VALU'][i], new['YDNTS_VALU'][i])
old['y'] = 0
old['x'] = 0
for j in range(0, len(old['XCNTS_VALU'])):
    old['y'][j], old['x'][j] = EPSG5181_to_wgs84(old['XCNTS_VALU'][j], old['YDNTS_VALU'][j])

In [4]:
# 폐업 레스토랑 y좌표정보 없는 행 삭제
deleteList_y =restaurant_closing[restaurant_closing['좌표정보(y)'] == '정보없음'].index
restaurant_closing.drop(deleteList_y, inplace=True)
deleteList_y

Int64Index([], dtype='int64')

In [5]:
# 폐업 레스토랑 X좌표정보 없는 행 삭제
deleteList_x =restaurant_closing[restaurant_closing['좌표정보(x)'] == '정보없음'].index
restaurant_closing.drop(deleteList_x, inplace=True)
deleteList_x

Int64Index([], dtype='int64')

In [50]:
# 인덱스 초기화
restaurant_closing.reset_index(inplace=True)

In [58]:
# 폐업 레스토랑 경도 위도 좌표값 변경
restaurant_closing['y'] = 0
restaurant_closing['x'] = 0
for j in range(0, len(restaurant_closing)):
    restaurant_closing['y'][j], restaurant_closing['x'][j] = EPSG5181_to_wgs84(restaurant_closing['좌표정보(x)'][j], restaurant_closing['좌표정보(y)'][j])

### 행정동 내부에 위치한지 확인
 - 신상권, 구상권, 오픈레스토랑, 폐업 레스토랑이 행정동의 멀티 폴리곤 또는 폴리곤 안에 속하는지 확인 후 입력

In [8]:
# 신상권
# polygon 행정동 내부에 위치한지 아닌지 true, false로 출력
dong['locate_new'] = 'NaN'
for i in range(0, len(dong)):
    poly = []
    # 폴리곤타입일때
    if dong['poly_type'][i] == 'Polygon':
        poly = Polygon(dong['polygon'][i])
        true_list = []
        for n in range(0, len(new)):
            y = new['x'][n] 
            x = new['y'][n]
            result = inside(poly,x,y) # 내부에 위치하면 true, 아니면 False
            if result:
                true_list.append(new['TRDAR_NO'][n])
        if true_list != []:
            dong['locate_new'][i] = true_list
            
    # 멀티폴리곤타입일때
    # 폴리곤이 2개 이상일때 각 해당하는 폴리곤 내부에 위치하는지 돌아가면서 확인
    if dong['poly_type'][i] == 'MultiPolygon':
        for j in range(0, len(dong['polygon'][i])):
            true_list = []
            poly = Polygon(dong['polygon'][i][j])
            for n in range(0, len(new)):
                y = new['x'][n] 
                x = new['y'][n]
                result = inside(poly,x,y)
                if result:
                    true_list.append(new['TRDAR_NO'][n])
        if true_list != []:
            dong['locate_new'][i] = true_list

2110877
2110869
2120157
2110216


In [9]:
# 구상권
# polygon 행정동 내부에 위치한지 아닌지 true, false로 출력
# 신상권과 동일
dong['locate_old'] = 'NaN'
for i in range(0, len(dong)):
    poly = []
    if dong['poly_type'][i] == 'Polygon':
        poly = Polygon(dong['polygon'][i])
        true_list = []
        for n in range(0, len(old)):
            y = old['x'][n] 
            x = old['y'][n]
            result = inside(poly,x,y)
            if result:
                true_list.append(old['TRDAR_CD'][n])
        if true_list != []:
            dong['locate_old'][i] = true_list
    if dong['poly_type'][i] == 'MultiPolygon':
        for j in range(0, len(dong['polygon'][i])):
            true_list = []
            poly = Polygon(dong['polygon'][i][j])
            for n in range(0, len(old)):
                y = old['x'][n] 
                x = old['y'][n]
                result = inside(poly,x,y)
                if result:
                    print(old['TRDAR_CD'][n])
                    true_list.append(old['TRDAR_CD'][n])
        if true_list != []:
            dong['locate_old'][i] = true_list

1000785
1000821
1000815
1000820
1001046
1001040
1000184
1000215


In [63]:
# 음식점 + 좌표값
# 음식점번호와 좌표값을 리스트형식으로 행정동 dataframe에 입력 
dong['locate_restaurant_total'] = '일치없음'
for i in range(0, len(dong)):
    poly = []
    if dong['poly_type'][i] == 'Polygon':
        poly = Polygon(dong['polygon'][i])
        true_list = []
        for n in range(0, len(restaurant)):
            y = restaurant['위도'][n]
            x = restaurant['경도'][n]
            result = inside(poly,x,y)
            if result:
                true_list.append((float(restaurant['위도'][n]), float(restaurant['경도'][n]), restaurant['상가업소번호'][n]))

        if true_list != []:
            dong['locate_restaurant_total'][i] = true_list
    if dong['poly_type'][i] == 'MultiPolygon':
        for j in range(0, len(dong['polygon'][i])):
            true_list = []
            poly = Polygon(dong['polygon'][i][j])
            for n in range(0, len(restaurant)):
                y = restaurant['위도'][n] 
                x = restaurant['경도'][n]
                result = inside(poly,x,y)
                if result:
                    true_list.append((float(restaurant['위도'][n]), float(restaurant['경도'][n]), restaurant['상가업소번호'][n]))
            if true_list != []:
                dong['locate_restaurant_total'][i] = true_list

In [252]:
# 음식점 + 음식점 종류
# 음식점번호와 음식점 카테고리 리스트형식으로 행정동 dataframe에 입력 
dong['locate_restaurant'] = '일치없음'
for i in range(0, len(dong)):
    poly = []
    if dong['poly_type'][i] == 'Polygon':
        poly = Polygon(dong['polygon'][i])
        true_list = []
        for n in range(0, len(restaurant)):
            y = restaurant['위도'][n] 
            x = restaurant['경도'][n]
            result = inside(poly,x,y)
            if result:
                true_list.append((restaurant['상가업소번호'][n], restaurant['상권업종중분류명'][n]))
        if true_list != []:
            dong['locate_restaurant'][i] = true_list
    if dong['poly_type'][i] == 'MultiPolygon':
        for j in range(0, len(dong['polygon'][i])):
            true_list = []
            poly = Polygon(dong['polygon'][i][j])
            for n in range(0, len(restaurant)):
                y = restaurant['위도'][n] 
                x = restaurant['경도'][n]
                result = inside(poly,x,y)
                if result:
                    true_list.append((restaurant['상가업소번호'][n], restaurant['상권업종중분류명'][n]))
        if true_list != []:
            dong['locate_restaurant'][i] = true_list

In [19]:
# 폐업음식점+음식점 종류
# 음식점번호와 음식점 카테고리 리스트형식으로 행정동 dataframe에 입력 
dong['locate_restaurant_closing_category'] = '일치없음'
for i in range(0, len(dong)):
    poly = []
    if dong['poly_type'][i] == 'Polygon':
        poly = Polygon(dong['polygon'][i])
        true_list = []
        for n in range(0, len(restaurant_closing)):
            y = restaurant_closing['x'][n] 
            x = restaurant_closing['y'][n]
            result = inside(poly,x,y)
            if result:
                true_list.append((restaurant_closing['번호'][n], restaurant_closing['category'][n]))
        if true_list != []:
            dong['locate_restaurant_closing_category'][i] = true_list
    if dong['poly_type'][i] == 'MultiPolygon':
        for j in range(0, len(dong['polygon'][i])):
            true_list = []
            poly = Polygon(dong['polygon'][i][j])
            for n in range(0, len(restaurant_closing)):
                y = restaurant_closing['x'][n] 
                x = restaurant_closing['y'][n]
                result = inside(poly,x,y)
                if result:
                    true_list.append((restaurant_closing['번호'][n], restaurant_closing['category'][n]))
        if true_list != []:
            dong['locate_restaurant_closing_category'][i] = true_list

In [8]:
# 폐업음식점+좌표값
# 음식점번호와 좌표값을 리스트형식으로 행정동 dataframe에 입력 
dong['locate_restaurant_closing'] = '일치없음'
for i in range(0, len(dong)):
    poly = []
    if dong['poly_type'][i] == 'Polygon':
        poly = Polygon(dong['polygon'][i])
        true_list = []
        for n in range(0, len(restaurant_closing)):
            y = restaurant_closing['x'][n] 
            x = restaurant_closing['y'][n]
            result = inside(poly,x,y)
            if result:
                true_list.append((float(restaurant_closing['x'][n]),float(restaurant_closing['y'][n]),restaurant_closing['번호'][n], ))
        if true_list != []:
            dong['locate_restaurant_closing'][i] = true_list
    if dong['poly_type'][i] == 'MultiPolygon':
        for j in range(0, len(dong['polygon'][i])):
            true_list = []
            poly = Polygon(dong['polygon'][i][j])
            for n in range(0, len(restaurant_closing)):
                y = restaurant_closing['x'][n] 
                x = restaurant_closing['y'][n]
                result = inside(poly,x,y)
                if result:
                    true_list.append((float(restaurant_closing['x'][n]),float(restaurant_closing['y'][n]),restaurant_closing['번호'][n]))
        if true_list != []:
            dong['locate_restaurant_closing'][i] = true_list

NameError: name 'inside' is not defined

### 밀집도
 - 두 좌표간의 거리차이를 통해 밀집도 레벨을 구간화
 - haversine 라이브러리 사용
 - 거리가 가까울 수록 값은 높아짐(밀집도)

#### 버전 1
 - 오픈업체 따로 폐업업체따로 계산하여 밀집도 산출
 - 문제점: 영업업체와 폐업 업체와 관계 비교가 어렵기 때문에 폐기

In [9]:
# 같은 동 내에 있는 두 가게간의 거리 산출 (영업업체)
# 속도개선
# AtoB_length = pd.DataFrame({'start_point' : [], 'goal_point' : [], 'length' : [], 'addr_code' : []})
length_list = []
col_list = ['start_point','goal_point','length','addr_code','length_level']
row_list = []
for i in range(0, len(dong)): 
    total = dong['locate_restaurant_total'][i]
    for j in range(0, len(total)):
        if total != '일치없음':
            start = (total[j][1], total[j][0])
            for k in range(j, len(total)):
                if total[j][2] != total[k][2]:
                    goal = (total[k][1], total[k][0])
                    length = haversine(start, goal, unit='m')
                    length_level = 0
                    if 0 <= length <= 100:
                        length_level = 4
                    if 100 <= length <= 200:
                        length_level = 3
                    if 200 <= length <= 300:
                        length_level = 2
                    if 300 <= length <= 400:
                        length_level = 1
                    if 400 <= length <= 500:
                        length_level = 0
                    temp = [total[j][2], total[k][2], length, dong['addr_code'][i], length_level]
                    row_list.append(temp)
AtoB_length_open = pd.DataFrame(row_list,columns=col_list)

In [10]:
# 같은 동 내에 있는 두 가게간의 거리 산출(폐업업체)
# 속도개선
# AtoB_length = pd.DataFrame({'start_point' : [], 'goal_point' : [], 'length' : [], 'addr_code' : []})
length_list = []
col_list = ['start_point','goal_point','length','addr_code','length_level']
row_list = []
for i in range(0, len(dong)): 
    total = dong['locate_restaurant_closing'][i]
    for j in range(0, len(total)):
        if total != '일치없음':
            start = (total[j][0], total[j][1])
            for k in range(j, len(total)):
                if total[j][2] != total[k][2]:
                    goal = (total[k][0], total[k][1])
                    length = haversine(start, goal, unit='m')
                    length_level = 0
                    if 0 <= length <= 100:
                        length_level = 4
                    if 100 <= length <= 200:
                        length_level = 3
                    if 200 <= length <= 300:
                        length_level = 2
                    if 300 <= length <= 400:
                        length_level = 1
                    if 400 <= length <= 500:
                        length_level = 0
                    temp = [total[j][2], total[k][2], length, dong['addr_code'][i], length_level]
                    row_list.append(temp)
AtoB_length_close = pd.DataFrame(row_list,columns=col_list)

In [72]:
# 영업중인 식당 또는 영업하지 않는 식당 중 한 곳만 데이터가 있는 컬럼 삭제
# 양쪽중에 한개의 데이터가 없으면 밀집도 비교가 어렵기 때문에 없는 동 삭제
dlist = dong[dong['locate_restaurant_closing'] == '일치없음'].index
dlist2 = dong[dong['locate_restaurant_total'] == '일치없음'].index
for d in dlist:
    dong.drop(int(d), inplace=True)
for d2 in dlist2:
    dong.drop(int(d2), inplace=True)
dong.drop(['level_0'], axis=1, inplace=True)
dong.reset_index(inplace=True)

#### 버전 2
 - 오픈업체와 폐업업체 합산한 밀집도 비교 + 폐업업체와 영업중인 업체의 밀집도 비교
 - 폐업업체를 포함한 해당하는 구역에 해당하는 밀집도 산출이 가능
 - 폐업업체와 영업중인 업체간의 밀집도 산출(폐업업체가 많은 지역은 어떠한 문제가 있을 것이다. - 유동인구가 적거나, 상권형성이 되지 않거나, 찾아오기 힘들다거나 등의 이유)

In [76]:
# 같은 동 내에 있는 두 가게간의 거리 산출(폐업업체+영업업체)
# 속도개선
# AtoB_length = pd.DataFrame({'start_point' : [], 'goal_point' : [], 'length' : [], 'addr_code' : []})
length_list = []
col_list = ['start_point','goal_point','length','addr_code','length_level']
row_list = []
for i in range(0, len(dong)): 
    total_open = dong['locate_restaurant_total'][i]
    total_close = dong['locate_restaurant_closing'][i]  
    total = total_open+total_close
    for j in range(0, len(total)):
        start = (total[j][0], total[j][1])
        for k in range(j, len(total)):
            if total[j][2] != total[k][2]:
                goal = (total[k][0], total[k][1])
                length = haversine(start, goal, unit='m')
                length_level = 0
                if 0 <= length <= 50:
                    length_level = 10
                if 50 <= length <= 100:
                    length_level = 9
                if 100 <= length <= 150:
                    length_level = 8
                if 150 <= length <= 200:
                    length_level = 7
                if 200 <= length <= 250:
                    length_level = 6
                if 250 <= length <= 300:
                    length_level = 5
                if 300 <= length <= 350:
                    length_level = 4
                if 350 <= length <= 400:
                    length_level = 3
                if 400 <= length <= 450:
                    length_level = 2
                if 450 <= length <= 500:
                    length_level = 1                    
                temp = [total[j][2], total[k][2], length, dong['addr_code'][i], length_level]
                row_list.append(temp)
total_density = pd.DataFrame(row_list,columns=col_list)

In [77]:
# 같은 동 내에 있는 두 가게간의 거리 산출(폐업업체와 영업중인업체와 차이)
# 속도개선
length_list = []
col_list = ['start_point','goal_point','length','addr_code','length_level']
row_list = []
for i in range(0, len(dong)): 
    close = dong['locate_restaurant_closing'][i]
    opens = dong['locate_restaurant_total'][i]
    for j in range(0, len(close)):
        if close != '일치없음':
            start = (close[j][0], close[j][1])
        else:
            pass
        for k in range(0, len(opens)):
            if opens != '일치없음':
                goal = (opens[k][0], opens[k][1])
                length = haversine(start, goal, unit='m')
                length_level = 0
                if 0 <= length <= 50:
                    length_level = 10
                if 50 <= length <= 100:
                    length_level = 9
                if 100 <= length <= 150:
                    length_level = 8
                if 150 <= length <= 200:
                    length_level = 7
                if 200 <= length <= 250:
                    length_level = 6
                if 250 <= length <= 300:
                    length_level = 5
                if 300 <= length <= 350:
                    length_level = 4
                if 350 <= length <= 400:
                    length_level = 3
                if 400 <= length <= 450:
                    length_level = 2
                if 450 <= length <= 500:
                    length_level = 1
                try:
                    temp = [close[j][2], opens[k][2], length, dong['addr_code'][i], length_level]
                except:
                    print(i)
                    print(j)
                    print(k)
                row_list.append(temp)
c_to_b_length = pd.DataFrame(row_list,columns=col_list)

In [149]:
'''
지나친 For문의 사용으로 속도가 너무 느림 
# 같은 동 내에 있는 두 가게간의 거리 산출
AtoB_length = pd.DataFrame({'start_point' : [], 'goal_point' : [], 'length' : [], 'addr_code' : []})
for i in range(0, 3):
    total = dong['locate_restaurant_total'][i]
    for j in range(0, len(total)):
        if total != '일치없음':
            start = (float(total[j][1]), float(total[j][0]))
            for k in range(j, len(total)):
                if total[j][2] != total[k][2]:
                    goal = (float(total[k][1]), float(total[k][0]))
                    length = haversine(start, goal, unit='m')
                    # 가게 간의 거리 300m 초과는 유의미하게 밀집하지 않다고 판단하여 배제
                    if length <= 300:
                        AtoB_length = AtoB_length.append(pd.DataFrame({'start_point' : int(total[j][2]), 'goal_point' : int(total[k][2]), 'length' : [length], 'addr_code' : int(dong['addr_code'][i])}), ignore_index = True)       
'''            

In [32]:
# dong데이터 저장
with open('../temp(dong).pkl', 'wb') as f:
    pickle.dump(dong, f, protocol=pickle.HIGHEST_PROTOCOL)

In [83]:
# 모든 업체간의 밀집도 피클저장
with open('../total_density.pkl', 'wb') as f:
    pickle.dump(total_density, f, protocol=pickle.HIGHEST_PROTOCOL)

In [84]:
# 오픈업체와 폐업업체간의 밀집도 피클저장
with open('../density_close_to_open.pkl', 'wb') as f:
    pickle.dump(c_to_b_length, f, protocol=pickle.HIGHEST_PROTOCOL)

In [127]:
# dong pkl 파일 저장
with open('../restaurant.pkl', 'wb') as f:
    pickle.dump(restaurant, f, protocol=pickle.HIGHEST_PROTOCOL)

In [126]:
# dong pkl 파일 저장
with open('../restaurant_closing.pkl', 'wb') as f:
    pickle.dump(restaurant_closing, f, protocol=pickle.HIGHEST_PROTOCOL)